<a href="https://colab.research.google.com/github/imran1386khan/Biobert-Training-Google-Colab/blob/master/2_Extract_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==2.0.0-alpha0

In [0]:
import os
import urllib.request

#Link to pre-trained BERT
urllib.request.urlretrieve('https://github.com/naver/biobert-pretrained/releases/download/v1.0-pubmed-pmc/biobert_v1.0_pubmed_pmc.tar.gz', 'BioBert.tar.gz')

if not os.path.exists('BioBertFolder'):
    os.makedirs('BioBertFolder')
    
import tarfile
tar = tarfile.open("BioBert.tar.gz")
tar.extractall(path='BioBertFolder/')
tar.close()

!wget https://github.com/re-search/DocProduct/archive/v0.2.0_dev.zip
!wget https://github.com/re-search/gpt2-estimator/archive/master.zip

!unzip master.zip
!unzip v0.2.0_dev.zip

!mv DocProduct-0.2.0_dev/* /content/
!mv gpt2-estimator-master/* /content/

!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2

!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

In [0]:
import numpy as np
import argparse
import cv2
import sys

import tensorflow as tf
# tf.compat.v1.enable_eager_execution()
tf.compat.v1.disable_eager_execution()
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
import argparse
import os
import requests

from docproduct.dataset import create_dataset_for_bert
from docproduct.models import MedicalQAModelwithBert
from docproduct.loss import qa_pair_loss, qa_pair_cross_entropy_loss
from docproduct.tokenization import FullTokenizer
from docproduct.metrics import qa_pair_batch_accuracy

In [0]:
import argparse
import os
from glob import glob

import pandas as pd
import numpy as np

from docproduct.predictor import QAEmbed

model_path='/content/drive/My Drive/DocAssistTraining/bertffn_crossentropy/bertffn/'
data_path='/content/data/'
output_path='/content/output.prkt'
pretrained_path='/content/BioBertFolder/biobert_v1.0_pubmed_pmc/'

!mkdir '/content/data'
!cp '/content/drive/My Drive/data/QandA_Dataset.csv' '/content/data/train_data.csv'

def read_all(data_path):
    glob_pattern = os.path.join(data_path, '*.csv')
    df_list = []
    for f in glob(glob_pattern):
        print('Reading {0}'.format(f))
        if os.path.basename(f) == 'healthtap_data_cleaned.csv':
            df = pd.read_csv(f, lineterminator='\n')
            df.columns = ['index', 'question', 'answer']
            df.drop(columns=['index'], inplace=True)
        else:
            df = pd.read_csv(f, encoding='utf8', lineterminator='\n')
        try:
            df.drop(columns=['question_bert', 'answer_bert'], inplace=True)
        except:
            pass
        df_list.append(df)
    return pd.concat(df_list, axis=0)


def train_data_to_embedding(model_path='models/bertffn_crossentropy/bertffn',
                            data_path='data/mqa_csv',
                            output_path='qa_embeddings/bertffn_crossentropy.zip',
                            pretrained_path='models/pubmed_pmc_470k/'):
    """Function to generate similarity embeddings for QA pairs.

    Input file format:
        question,answer
        my eyes hurts, go see a doctor

    Keyword Arguments:
        model_path {str} -- Similarity embedding model path (default: {'models/bertffn_crossentropy/bertffn'})
        data_path {str} -- CSV data path (default: {'data/mqa_csv'})
        output_path {str} -- Embedding output path (default: {'qa_embeddings/bertffn_crossentropy.zip'})
        pretrained_path {str} -- Pretrained BioBert model path (default: {'models/pubmed_pmc_470k/'})
    """
    if os.path.basename(model_path) == 'ffn':
        ffn_weight_file = model_path
    else:
        ffn_weight_file = None

    if os.path.basename(model_path) == 'bertffn':
        bert_ffn_weight_file = model_path
    else:
        bert_ffn_weight_file = None
    embeder = QAEmbed(
        pretrained_path=pretrained_path,
        ffn_weight_file=ffn_weight_file,
        bert_ffn_weight_file=bert_ffn_weight_file
    )
    qa_df = read_all(data_path)
    qa_df.dropna(inplace=True)
    qa_vectors = embeder.predict(
        questions=qa_df.question.tolist(),
        answers=qa_df.answer.tolist())

    q_embedding, a_embedding = np.split(qa_vectors, 2, axis=1)
    qa_df['Q_FFNN_embeds'] = np.squeeze(q_embedding).tolist()
    qa_df['A_FFNN_embeds'] = np.squeeze(a_embedding).tolist()
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    qa_df.to_parquet(output_path, index=False)

In [0]:
train_data_to_embedding(model_path=model_path,
                            data_path=data_path,
                            output_path=output_path,
                            pretrained_path=pretrained_path)